In [1]:
import tiktoken
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

from gpt_model import GPTModel
from clean_gutenberg_text import clean_gutenberg_text
from train_sae import train_sae

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using {device} device.")

Using mps device.


In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 256,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.2,
    "qkv_bias": False,
    "device": device,
}

In [4]:
model = GPTModel(GPT_CONFIG_124M)
checkpoint = torch.load("model_and_optimizer_5.pth", weights_only=True)

model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)
model.eval();

In [5]:
tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
import re

def load_and_clean_text(file_path):
    """
    Loads a text file and splits it into sentences while cleaning the text.
    
    Args:
    - file_path (str): Path to the text file.
    
    Returns:
    - list: A list of cleaned sentences from the book.
    """
    
    text = clean_gutenberg_text(file_path)

    # Split text into sentences (simple heuristic using punctuation)
    sentences = re.split(r"(?<=[.!?])\s+", text)

    # Remove very short or long sentences
    sentences = [s.strip() for s in sentences if 5 < len(s.split()) < 50]

    return sentences

In [7]:
directory="original_texts/"
dataset = []

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        sentences = load_and_clean_text(os.path.join(directory, filename))
        dataset += sentences

In [8]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def get_token_embeddings(text, model, tokenizer, layers=[6, 12]):
    """
    Extracts token embeddings from specified transformer layers.

    Args:
    - text (str): Input text.
    - model: Custom GPT model.
    - tokenizer: tiktoken encoding object.
    - layers (list): Transformer layers to extract embeddings from.

    Returns:
    - dict: Layer-wise token embeddings {layer_number: embeddings}
    """

    input_ids = text_to_token_ids(text, tokenizer).to(device)

    with torch.no_grad():
        _, hidden_states = model(input_ids, output_hidden_states=True)

    embeddings = {}
    for layer in layers:
        if layer - 1 < len(hidden_states):
            embeddings[layer] = hidden_states[layer - 1].squeeze(0).cpu().numpy()
        else:
            print(f"⚠️ Warning: Layer {layer} is out of range (max index {len(hidden_states) - 1})")

    return embeddings

In [9]:
layer6_embeddings = []
layer12_embeddings = []

for sentence in dataset:
    embeddings = get_token_embeddings(sentence, model, tokenizer)
    layer6_embeddings.append(embeddings[6])
    layer12_embeddings.append(embeddings[12])

# Convert to NumPy and flatten tokens into dataset
layer6_embeddings = np.vstack(layer6_embeddings)
layer12_embeddings = np.vstack(layer12_embeddings)

os.makedirs("sae_data", exist_ok=True)
np.save("sae_data/layer6_embeddings.npy", layer6_embeddings)
np.save("sae_data/layer12_embeddings.npy", layer12_embeddings)

print("Saved token embeddings:")
print(f"Layer 6: {layer6_embeddings.shape}")
print(f"Layer 12: {layer12_embeddings.shape}")

Saved token embeddings:
Layer 6: (280474, 768)
Layer 12: (280474, 768)


In [9]:
layer6_embeddings = np.load("sae_data/layer6_embeddings.npy")
layer12_embeddings = np.load("sae_data/layer12_embeddings.npy")

In [10]:
train_losses = []
val_losses = []
global train_losses, val_losses

train_sae(layer6_embeddings, "sae_layer6_1.pth", train_losses=[],
          val_losses=val_losses, epochs=500, device=device)

/Users/mariam/opt/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/500], Train Loss: 2.3618, Val Loss: 2.2800
Epoch [2/500], Train Loss: 2.2735, Val Loss: 2.2984
Epoch [3/500], Train Loss: 2.2630, Val Loss: 2.2754
Epoch [4/500], Train Loss: 2.2570, Val Loss: 2.2771
Epoch [5/500], Train Loss: 2.2525, Val Loss: 2.2562
Epoch [6/500], Train Loss: 2.2502, Val Loss: 2.2787
Epoch [7/500], Train Loss: 2.2485, Val Loss: 2.2574
Epoch [8/500], Train Loss: 2.2471, Val Loss: 2.2740
Epoch [9/500], Train Loss: 2.2461, Val Loss: 2.2814
Epoch [10/500], Train Loss: 2.2452, Val Loss: 2.2925
Epoch [11/500], Train Loss: 2.2444, Val Loss: 2.2744
Epoch [12/500], Train Loss: 2.2260, Val Loss: 2.2265
Epoch [13/500], Train Loss: 2.2237, Val Loss: 2.2189
Epoch [14/500], Train Loss: 2.2233, Val Loss: 2.2273
Epoch [15/500], Train Loss: 2.2230, Val Loss: 2.2203
Epoch [16/500], Train Loss: 2.2226, Val Loss: 2.2392
Epoch [17/500], Train Loss: 2.2220, Val Loss: 2.2464
Epoch [18/500], Train Loss: 2.2219, Val Loss: 2.2346
Epoch [19/500], Train Loss: 2.2212, Val Loss: 2.2189
Ep

In [11]:
train_losses = []
val_losses = []
global train_losses, val_losses

train_sae(layer12_embeddings, "sae_layer12.pth", train_losses=[],
          val_losses=val_losses, epochs=500, device=device)

Epoch [1/500], Train Loss: 5.9004, Val Loss: 5.7176
Epoch [2/500], Train Loss: 5.7059, Val Loss: 5.6882
Epoch [3/500], Train Loss: 5.6882, Val Loss: 5.7546
Epoch [4/500], Train Loss: 5.6805, Val Loss: 5.7202
Epoch [5/500], Train Loss: 5.6752, Val Loss: 5.7325
Epoch [6/500], Train Loss: 5.6718, Val Loss: 5.6864
Epoch [7/500], Train Loss: 5.6699, Val Loss: 5.6657
Epoch [8/500], Train Loss: 5.6674, Val Loss: 5.6849
Epoch [9/500], Train Loss: 5.6654, Val Loss: 5.7040
Epoch [10/500], Train Loss: 5.6632, Val Loss: 5.7012
Epoch [11/500], Train Loss: 5.6616, Val Loss: 5.7231
Epoch [12/500], Train Loss: 5.6603, Val Loss: 5.6998
Epoch [13/500], Train Loss: 5.6603, Val Loss: 5.6749
Epoch [14/500], Train Loss: 5.6296, Val Loss: 5.5976
Epoch [15/500], Train Loss: 5.6270, Val Loss: 5.6285
Epoch [16/500], Train Loss: 5.6266, Val Loss: 5.6242
Epoch [17/500], Train Loss: 5.6256, Val Loss: 5.6264
Epoch [18/500], Train Loss: 5.6253, Val Loss: 5.6167
Epoch [19/500], Train Loss: 5.6253, Val Loss: 5.5838
Ep